In [1]:
import sys
import json
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [3]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')


In [4]:
sm = stepmodel.read_zippedpickles(model_folder +'los')

zones: 100%|██████████| 27/27 [00:03<00:00,  8.72it/s]          


In [5]:
sm.zones

,density,population,numWork,geometry
zone_0,14881.3,17746,11063,"POLYGON ((248627.018 3283999.200, 248603.383 3..."
zone_1,2268.8,17904,11301,"POLYGON ((277883.614 3315713.790, 277816.536 3..."
zone_2,7890.2,20528,12971,"POLYGON ((298936.545 3262707.637, 297507.152 3..."
zone_3,2418.2,15187,8670,"POLYGON ((262501.025 3340907.093, 261625.727 3..."
zone_4,16441.9,23824,13447,"POLYGON ((263217.335 3286655.111, 263226.764 3..."
...,...,...,...,...
zone_495,4360.0,17132,10032,"POLYGON ((279621.252 3299078.842, 278655.722 3..."
zone_496,272.8,2636,1549,"POLYGON ((255580.979 3359468.270, 254044.672 3..."
zone_497,7877.0,13450,8393,"POLYGON ((262350.051 3294663.488, 259620.245 3..."
zone_498,2378.2,12637,7491,"POLYGON ((315962.178 3253070.662, 313195.653 3..."


In [6]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False) 
motor_rate = var['step_distribution']['motor_rate']
pt_rate = (1-motor_rate)


In [7]:

sm.zones['population'].sum()

7149642

In [8]:
sm.zones['emission'] =  sm.zones['population']
sm.zones['attraction'] =  sm.zones['numWork'] ####### TO CHANGE: WORK COLUMN

car = sm.copy()
car.zones['attraction']= car.zones['attraction']
car.zones['emission'] *= motor_rate  #car.zones['emission'] = (car.zones['emission'] * car.zones['motorization_rate']).fillna(0)

pt_captive = sm.copy()
pt_captive.zones['emission']= pt_captive.zones['emission']
pt_captive.zones['emission'] *= pt_rate

In [9]:
imp_matrix = car.car_los[ 
    ['origin', 'destination', 'time']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, 120)

car.step_distribution(deterrence_matrix=imp_matrix) 


In [10]:
imp_matrix

time                                            \
destination        zone_0        zone_1       zone_10      zone_100   
origin                                                                
zone_0         120.000000   3445.684922   2020.607366   3099.640056   
zone_1        3410.705945    120.000000   3392.890681   2679.658128   
zone_10       2025.797768   3442.219786    120.000000   2669.899368   
zone_100      3072.918457   2700.549512   2650.722582    120.000000   
zone_101     12558.721104  11220.215261  12540.905839  11268.545963   
...                   ...           ...           ...           ...   
zone_95      15596.676345  16786.916376  16672.598891  16830.803679   
zone_96       2191.367492   4831.428431   2739.417140   4355.785650   
zone_97       3520.508644   4710.748676   4596.431191   4754.635978   
zone_98      37267.057205  39907.118144  38047.920654  39664.289164   
zone_99       4029.550821   5219.790852   5105.473367   5263.678155   

                                                                     \
destination      zone_101      zone_102      zone_103      zone_104   
origin                                                                
zone_0       12542.204179   2708.945190   3356.558840   1432.576731   
zone_1       11200.796616   3871.758320   2877.097996   2496.183887   
zone_10      12526.188286   3802.394934   2926.818152   1597.392924   
zone_100     11177.427305   3954.058145   1522.076913   2316.680118   
zone_101       120.000000  12620.459577  11353.854191  11644.199045   
...                   ...           ...           ...           ...   
zone_95      25159.554265  13703.041966  16981.574780  15399.321111   
zone_96      13927.947688   4037.439475   4612.704434   2846.580647   
zone_97      13263.286701   1626.874266   4905.407080   3323.153411   
zone_98      49003.637401  38769.345933  39921.207948  37922.270360   
zone_99      13446.794771   2055.782075   5414.449256   3832.195587   

                                         ...                              \
destination      zone_105      zone_106  ...       zone_90       zone_91   
origin                                   ...                               
zone_0        4323.567656  10071.853923  ...   3504.714896  20722.337132   
zone_1        4711.210486   9484.976564  ...   4196.883177  20893.545408   
zone_10       5133.452501  10699.970854  ...   4557.742940  21348.966929   
zone_100      4946.884284   9635.265289  ...   4650.719800  21166.052352   
zone_101     11748.340706  15681.287693  ...  12482.387074  27967.508774   
...                   ...           ...  ...           ...           ...   
zone_95      15796.583320  22498.335319  ...  14981.308689  32560.367895   
zone_96       5692.883597  11457.597431  ...   4833.209181  22108.080641   
zone_97       3900.315755  10602.067754  ...   3085.041124  20664.100330   
zone_98      40566.018915  46533.287144  ...  39565.115640  57183.770354   
zone_99       4083.823826  10785.575825  ...   3268.549194  20847.608401   

                                                                     \
destination       zone_92       zone_93       zone_94       zone_95   
origin                                                                
zone_0        6820.146782   4907.599345   4770.051933  15602.348637   
zone_1        9435.154496   5120.668471   6665.632525  16765.161767   
zone_10       7611.647868   4482.234308   4077.141737  16695.798380   
zone_100      9200.198890   2711.628751   5735.111679  16847.461591   
zone_101     18583.169654  13653.647013  15779.046289  25181.502884   
...                   ...           ...           ...           ...   
zone_95      21052.058615  19236.234591  19422.043458    120.000000   
zone_96       6496.290703   6194.683465   5094.175278  16930.842922   
zone_97       8975.890915   7160.066891   7345.875758  13130.937629   
zone_98      37081.574249  41474.027485  40269.315893  51427.953240   
zone_99       9377.327466   7669.109067   7

In [11]:
imp_matrix = pt_captive.car_los[ 
    ['origin', 'destination', 'time']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, 120)

In [12]:
imp_matrix

time                                            \
destination        zone_0        zone_1       zone_10      zone_100   
origin                                                                
zone_0         120.000000   3445.684922   2020.607366   3099.640056   
zone_1        3410.705945    120.000000   3392.890681   2679.658128   
zone_10       2025.797768   3442.219786    120.000000   2669.899368   
zone_100      3072.918457   2700.549512   2650.722582    120.000000   
zone_101     12558.721104  11220.215261  12540.905839  11268.545963   
...                   ...           ...           ...           ...   
zone_95      15596.676345  16786.916376  16672.598891  16830.803679   
zone_96       2191.367492   4831.428431   2739.417140   4355.785650   
zone_97       3520.508644   4710.748676   4596.431191   4754.635978   
zone_98      37267.057205  39907.118144  38047.920654  39664.289164   
zone_99       4029.550821   5219.790852   5105.473367   5263.678155   

                                                                     \
destination      zone_101      zone_102      zone_103      zone_104   
origin                                                                
zone_0       12542.204179   2708.945190   3356.558840   1432.576731   
zone_1       11200.796616   3871.758320   2877.097996   2496.183887   
zone_10      12526.188286   3802.394934   2926.818152   1597.392924   
zone_100     11177.427305   3954.058145   1522.076913   2316.680118   
zone_101       120.000000  12620.459577  11353.854191  11644.199045   
...                   ...           ...           ...           ...   
zone_95      25159.554265  13703.041966  16981.574780  15399.321111   
zone_96      13927.947688   4037.439475   4612.704434   2846.580647   
zone_97      13263.286701   1626.874266   4905.407080   3323.153411   
zone_98      49003.637401  38769.345933  39921.207948  37922.270360   
zone_99      13446.794771   2055.782075   5414.449256   3832.195587   

                                         ...                              \
destination      zone_105      zone_106  ...       zone_90       zone_91   
origin                                   ...                               
zone_0        4323.567656  10071.853923  ...   3504.714896  20722.337132   
zone_1        4711.210486   9484.976564  ...   4196.883177  20893.545408   
zone_10       5133.452501  10699.970854  ...   4557.742940  21348.966929   
zone_100      4946.884284   9635.265289  ...   4650.719800  21166.052352   
zone_101     11748.340706  15681.287693  ...  12482.387074  27967.508774   
...                   ...           ...  ...           ...           ...   
zone_95      15796.583320  22498.335319  ...  14981.308689  32560.367895   
zone_96       5692.883597  11457.597431  ...   4833.209181  22108.080641   
zone_97       3900.315755  10602.067754  ...   3085.041124  20664.100330   
zone_98      40566.018915  46533.287144  ...  39565.115640  57183.770354   
zone_99       4083.823826  10785.575825  ...   3268.549194  20847.608401   

                                                                     \
destination       zone_92       zone_93       zone_94       zone_95   
origin                                                                
zone_0        6820.146782   4907.599345   4770.051933  15602.348637   
zone_1        9435.154496   5120.668471   6665.632525  16765.161767   
zone_10       7611.647868   4482.234308   4077.141737  16695.798380   
zone_100      9200.198890   2711.628751   5735.111679  16847.461591   
zone_101     18583.169654  13653.647013  15779.046289  25181.502884   
...                   ...           ...           ...           ...   
zone_95      21052.058615  19236.234591  19422.043458    120.000000   
zone_96       6496.290703   6194.683465   5094.175278  16930.842922   
zone_97       8975.890915   7160.066891   7345.875758  13130.937629   
zone_98      37081.574249  41474.027485  40269.315893  51427.953240   
zone_99       9377.327466   7669.109067   7

In [13]:
pt_captive.step_distribution(deterrence_matrix=imp_matrix)

In [14]:
sm.volumes = pd.merge(
    car.volumes,
    pt_captive.volumes,
    on=['origin', 'destination'],
    suffixes=[ '_car_owner', '_pt_captive']
).rename(columns={'volume_car_owner': 'car_owner', 'volume_pt_captive': 'pt_captive'})
sm.volumes

,origin,destination,car_owner,pt_captive
0,zone_0,zone_0,1.893451,0.473363
1,zone_0,zone_1,43.147247,10.786812
2,zone_0,zone_2,33.700750,8.425187
3,zone_0,zone_3,32.733456,8.183364
4,zone_0,zone_4,59.190368,14.797592
...,...,...,...,...
249995,zone_499,zone_495,16.103782,4.025946
249996,zone_499,zone_496,3.562442,0.890610
249997,zone_499,zone_497,8.168496,2.042124
249998,zone_499,zone_498,14.166690,3.541672


In [15]:
sm.to_zippedpickles(model_folder +'distribution')


volumes: 100%|██████████| 28/28 [00:10<00:00,  2.76it/s]        
